# Nuevas funciones para el procesamiento con Spacy: funciones_spacy
Este *notebook* en *Python* contiene una serie de funciones que sintetizan en una sola orden el código de *SpaCy* necesario para realizar determinadas tareas repetitivas. Por ejemplo, serializar o revertir la serialización de una instancia `Doc()`.

## Forma de uso
Mi recomendación no es copiar cada función cuando sea necesaria, sino, ejecutar este *notebook* en cada uno de los demás en que usamos el código desarrallado en estas prácticas. Debe hacerse de la siguiente forma:
- Usamos el comando mágico `%run`.
- Lo hacemos escribiendo en una celda `%run <path of the notebook that has to be run>`.
- Por ejemplo, `%run './funciones_Spacy.ipynb'`.
- Más información en [este link](https://docs.qubole.com/en/latest/user-guide/notebooks-and-dashboards/notebooks/jupyter-notebooks/running-jupy-notebooks.html)

## Lista de funciones que contiene este *notebook*
- `cuenta_palabras_max`: esta función toma un archivo txt alojado en el disco y devuelve el número máximo de palabras que contiene una frase del documento.
- `revertir_serializacion`: toma un archivo binario con un documento serializado y lo convierte de nuevo en una variable de *Python* como una instancia `Doc()`.
- `serializacion`: toma un documento y lo guarda en el disco como un archivo de datos, es decir, un archivo binario.
- `verifica_matcher`: esta función recibe un patrón para el `PhraseMatcher` de SpaCy, también variables de ayuda para crear una etiqueta de Span útil para la automatización de la extracción.

## Contador de palabras máximas: cuenta_palabras_max
Esta función toma un archivo txt alojado en el disco y devuelve el número máximo de palabras que contiene una frase del documento.

### Sintaxis
`numero = cuenta_palabras_max(direcciontxt)`
### Valor de los parámetros
| Parámetro   | Descripción |
|:----------- |:-------------|
| direcciontxt| Es la dirección donde está el txt|
### Código

In [ ]:
def cuenta_palabras_max(direcciontxt):
    """
    Esta función toma un archivo txt alojado en el disco y devuelve el número máximo de palabras que contiene una frase del documento.
    Parámetros: `direcciontxt`: dirección onde está el txt.
    Devuelve: `num`: es el número buscado.
    """
    
    # Inicializamos la variable numero
    numero = -1
    
    # Abrimos el documento y recorremos sus líneas
    with open(direcciontxt, encoding='utf-8') as f:
        
        # Recorremos sus líneas
        for linea in f:
            
            # Rompemos cada línea en un espacio con split() y contamos el número de palabras con len
            lista = linea.split(" ")
            longitud = len(lista)
            
            # Comparamos `longitud` con `numero` y guardamos el valor mayor.
            if longitud > numero:
                numero = longitud
    
    return numero



## Revertir la serialización:  `revertir_serializacion`
Esta función toma del disco una instancia un binario que contiene una instancia `Doc()` serializada por *Spacy* y lo devuelve a una instancia `Doc()` en una variable de Python.
### Sintaxis
`doc = revertir_serializacion(direccionbin, nlp, extensiones)`
### Valor de los parámetros
| Parámetro   | Descripción |
|:----------- |:-------------|
| direccionbin| Es la dirección donde está el binario.|
| nlp         | Es la función pipeline con la que procesamos el binario. |
| extensiones   | Si el binario contiene etiquetas definidas por el usuario, debe darse información es éstas para poder rescatarlas.|
### Ejemplo de uso
Véase un ejemplo al final del *notebook* `B_Añadir_Etiquetas.ipynb`
### Código

In [ ]:
def revertir_serializacion(direccionbin, nlp, extensiones=None):
    """
    Esta función toma del disco una instancia un binario que contiene una instancia Doc() serializada por Spacy y lo 
    devuelve a una instancia Doc() en una variable de Python. Si el binario tiene extensiones elaboradas por el usuario
    y no se cargan con set_extension, como se hace aquí, no podrán ser leídas posteriormente.
    Parámetros: `direccionbin`: direción donde está el archivo binario,
                `nlp`: función de procesamiento de lenguaje natural de SpaCy.
                `extensiones`: diccionario que recoge la información de las extensiones elaboradas por el usuario y 
                    que el archivo binario debe tener registradas.
    Resultado: `doc`: instancia de Doc de SpaCy.
    """
    print('Nota: esta función requiere los siguientes módulo o funciones:\n' ,
          'import spacy\n',
          'from spacy.tokens import Doc\n',
          'from spacy.tokens import DocBin\n',
          'Si no los usa se producirá un error.')
    
    doc_bin = DocBin().from_disk(direccionbin)
    doc = list(doc_bin.get_docs(nlp.vocab))[0]
    
    if extensiones != None:
        for nombre, dic in extensiones.items():
            if dic['tipo'] == 'span':
                Span.set_extension(nombre, default=dic['default'], force=True)
    
    return doc



## Serializacion: `serializacion`
Esta función toma una instancia `Doc()` de *SpaCy*, lo serializa y guarda en el disco en la dirección que se le da a la función.
### Sintaxis
`serializacion( doc, direccionbin)`
### Valor de los parámetros
| Parámetro | Descripción |
|:-----------|:-------------|
| doc       | Es una instancia `Doc()` de *SpaCy* |
| direccionbin| Es la dirección donde se guardará el binario|
### Código

In [ ]:
def serializacion(doc, direccionbin):
    """
    Esta función toma una instancia `Doc()` de SpaCy, lo serializa y guarda en el disco en la dirección que se le da a la 
    función.
    Parámetros: `doc` (instancia Doc()) y `direccionbin` (direción para guardar el archivo binario).
    """
    print('Nota: esta función requiere los siguientes módulo o funciones:\n' ,
          'import spacy\n',
          'from spacy.tokens import DocBin\n',
          'Si no los usa se producirá un error.')
    
    # Creamos un DocBin
    doc_bin = DocBin(store_user_data=True)
    doc_bin.add(doc)
    
    # Guardamos en el disco.
    doc_bin.to_disk(direccionbin)
    
    

## Crear etiquetas usando patrones: `verifica_matcher`
Esta función recibe un patrón para el `PhraseMatcher` de SpaCy, también variables de ayuda para crear una etiqueta de Span útil para la automatización de la extracción.
### Sintaxis
`serializacion( doc, direccionbin)`
### Valor de los parámetros
| Parámetro | Descripción |
|:-----------|:-------------|
| nlp       | Es el pipeline de *SpaCy* |
| doc       | Es una instancia `Doc()` de *SpaCy* |
| nombre_extensión| Es el nombre de la etiqueta a crear|
| PhraseM   | Si se quiere aplicar el PhraseMatcher, se coloca aquí una lista de frases exactas a identificar|
| ayuda     | Esta variable sirve para el etiquetado autómatico. Es un diccionario cuyas claves es el texto a identificar y cuyo valor es la etiqueta a asignar|
| patrón    | Si no se usa el PhraseMatcher, debe usarse un patrón de búsqueda de SpaCy|

### Código

In [ ]:
def verifica_matcher(nlp, doc, nombre_extensión, PhraseM=False, ayuda=None, quitarPM=False, patrón=None,):
    """
    Esta función recibe un patrón para el `matcher` de SpaCy y una etiqueta a crear o
    rellenar interactivamente sobre los resultados del matcher.
    """
    print('Nota: esta función requiere los siguientes módulo o funciones:\n' ,
          'from spacy.matcher import Matcher, PhraseMatcher\n',
          'from spacy.tokens import Span\n',
          'Si no los usa se producirá un error.')
    
    if PhraseM == False:
        # Creamos y aplicamos el Matcher.
        matcher = Matcher(nlp.vocab)
        matcher.add("patrón", patrón)
        matches = matcher(doc)
        matches_quitar = None
        
    elif PhraseM != False and quitarPM != False:
        # Creamos y aplicamos el Matcher.
        matcher = PhraseMatcher(nlp.vocab)
        matcher.add("patrón", nlp.pipe(quitarPM))
        matches_quitar = matcher(doc)
        # Creamos y aplicamos el PhraseMatcher.
        matcher = PhraseMatcher(nlp.vocab)
        matcher.add("patrón", nlp.pipe(PhraseM))
        matches = matcher(doc)
        
    else:
        # Creamos y aplicamos el PhraseMatcher.
        matcher = PhraseMatcher(nlp.vocab)
        matcher.add("patrón", nlp.pipe(PhraseM))
        matches = matcher(doc)
        matches_quitar = None
        
    for elem in range(0,len(matches)):
        matches.append(matches[0][1:])
        matches.pop(0)
        
    if matches_quitar != None:
        
        for elem in range(0,len(matches_quitar)):
            matches_quitar.append(matches_quitar[0][1:])
            matches_quitar.pop(0)
            
        # Quitamos las coincidencias que no nos interesa
        lista_quitar = []
        for start, end in matches:
            
            for elem in matches_quitar:
                startq, endq = elem
                
                if startq <= start <= end <= endq:
                    lista_quitar.append((start, end))
                    
        for elem in lista_quitar:
            matches.pop(matches.index(elem))
            
    Span.set_extension(nombre_extensión, default=False, force=True)
    dic = {}
    # Ahora recorremos `matches` y lo mostramos junto con la frase donde lo encontramos
    l = len(matches)
    print('Se han encontrado', l, 'concordancias con el patrón.')
    i = 0
    j = []
    while i < l:
        start, end = matches[i]
        match = doc[start:end]
        matchtxt = match.text.upper()
        line = match.sent
        texto = line.text
        i += 1
        ayuda_usada = True
        
        if ayuda != None:
            info = ayuda.get(texto.upper())
            
            if info == None and ('EL ' in texto or 'LA ' in texto or 'LUGAR DE ' in texto):
                texto2 = texto.upper().replace('EL ', '').replace('LA ', '').replace('LUGAR DE ', '')
                info2 = ayuda.get(texto2)
                
                if info2 == None:
                    ayuda_usada = False
                    j.append(i - 1)
                    
                else:
                    line._.set(nombre_extensión, info2)
                    
            elif info == None and not ('EL ' in texto or 'LA ' in texto or 'LUGAR DE ' in texto):
                ayuda_usada = False
                j.append(i - 1)
                
            else:
                
                if ';' in info:
                    ayuda_usada = False
                    j.append(i - 1)
                    
                else:
                    line._.set(nombre_extensión, info)
                    
        coincidencia = ayuda.get(matchtxt)
        if ayuda_usada == False and coincidencia != None:
            print('Concordancia número', str(i-1) + '.', 'id de los tokens: ', [start, end])
            print('\nDígame "sí" en caso de querer usar', coincidencia, 'como etiqueta de', match.text + '\n en la línea "' + texto + '".\n',
                  'De lo contrario, escriba la etiqueda deseada.\n',
                  'Escriba "v", si quiere volverl al match anterior.\n',
                  'Escriba "parar", si quiere pasar al siguiente match.')
            etiqueta = input('Escriba aquí: ')
            print('\n')
            
            if etiqueta.lower() == 'parar':
                continue
                
            elif etiqueta.lower() == 'sí' or etiqueta.lower() == 'si':
                line._.set(nombre_extensión, coincidencia)
                
            elif etiqueta.lower() == 'v':
                j.pop(-1)
                i = j[-1]
                continue
                
            else:
                line._.set(nombre_extensión, etiqueta)
                
        elif ayuda_usada == False and coincidencia == None:
            print('Concordancia número', str(i-1) + '.', 'id de los tokens: ', [start, end])
            print('\nDígame la etiqueta de', match.text + ' en la línea "' + texto + '".\n',
                  'Escriba "v", si quiere volverl al match anterior.\n', 
                  'Escriba "parar", si quiere pasar al siguiente match.')
            etiqueta = input('Escriba aquí: ')
            print('\n')
            
            if etiqueta.lower() == 'parar':
                continue
                
            elif etiqueta.lower() == 'v':
                j.pop(-1)
                i = j[-1]
                continue
                
            else:
                line._.set(nombre_extensión, etiqueta)
                
        dic[start] = {'start': start, 'end': end, 'id': [line.sent.start, line.sent.end], 'etiqueta': line._.get(nombre_extensión)}
        
    return dic

